In [ ]:
pip install pickle5

Note: you may need to restart the kernel to use updated packages.


In [206]:
import pickle
import pickle5
import swifter
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Loading Data in and Preprocessing Abstractive and Extractive

In [412]:
with open ('extractive_summaries.pkl', 'rb') as fp:
    extractive = pickle.load(fp)

In [286]:
with open ('abs_sum_1000.pkl', 'rb') as fp:
    abstractive_1000 = pickle5.load(fp)

In [209]:
original_reading_in = pd.read_csv('narrs_cleaned_sent_tokenized.csv')

In [210]:
original_uncleaned = original_reading_in['Consumer complaint narrative']
original_uncleaned[0]

"['transworld systems inc. is trying to collect a debt that is not mine, not owed and is inaccurate.']"

In [211]:
def string_to_list(doc):
  import ast
  lists = ast.literal_eval(doc) #Resolve unicode errors 
  return lists

In [212]:
original_cleaned1 = original_uncleaned.swifter.apply(string_to_list)

In [213]:
original_cleaned1[0]

['transworld systems inc. is trying to collect a debt that is not mine, not owed and is inaccurate.']

In [214]:
def lowering(doc):
    lowered = " ".join([sentence.lower() for sentence in doc])
    return lowered

In [215]:
original_cleaned2 = original_cleaned1.progress_apply(lowering)

In [216]:
original_cleaned2[0]

'transworld systems inc. is trying to collect a debt that is not mine, not owed and is inaccurate.'

In [217]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

def stop_free(doc): 
  stop_words = set(stopwords.words('english'))
  stop_rem = " ".join([word for word in doc.split() if word not in stop_words])
  return stop_rem

[nltk_data] Downloading package punkt to /Users/kagenlim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kagenlim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [218]:
original_cleaned3 = original_cleaned2.progress_apply(stop_free)

In [219]:
original_cleaned3[0]

'transworld systems inc. trying collect debt mine, owed inaccurate.'

In [287]:
len(extractive[0:1000])

1000

In [288]:
extractive_1000 = pd.Series(extractive[0:1000])

In [289]:
len(abstractive_1000)

1000

In [290]:
def full_stop_space(doc):
  import re
  rx = r"\.(?=\S)"
  result = re.sub(rx, ". ", doc)
  return result

In [291]:
extractive_1000[1]

#full stop is a problem

'The company does not have the right to harass me at work and I want this to stop.It is extremely distracting to be told 5 times a day that I have a call from this collection agency while at work.'

In [292]:
extractive_1000 = extractive_1000.swifter.apply(full_stop_space)

In [293]:
extractive_1000[1]

#full stop is no longer a problem

'The company does not have the right to harass me at work and I want this to stop. It is extremely distracting to be told 5 times a day that I have a call from this collection agency while at work.'

In [294]:
def odd_tokens_free(doc):
  import re
  processed = re.sub("(xx|xxxx|XX|XXXX|XX/XX/XXXX)","",doc)
  return processed

In [295]:
extractive_1000 = extractive_1000.swifter.apply(odd_tokens_free)

In [296]:
abstractive_1000['Consumer complaint narrative'] = abstractive_1000['Consumer complaint narrative'].swifter.apply(odd_tokens_free)

In [297]:
abstractive_1000.rename(columns = {'Consumer complaint narrative':'abstractive'}, inplace = True) 

In [298]:
abstractive_1000.head()

,abstractive
0,transworld systems is trying to collect a debt...
1,the company listed in this complaint does not ...
2,each violation is a statutory cost of $1000.00...
3,experian failed to send me a copy of the verif...
4,a bankruptcy and two accounts show discrepanci...


In [299]:
def lowering(doc):
    lowered = "".join([sentence.lower() for sentence in doc])
    return lowered

In [300]:
extractive_1000 = extractive_1000.swifter.apply(lowering)

In [301]:
extractive_1000

0                                                       
1      the company does not have the right to harass ...
2      violations committed against me include but no...
3      previously, on //, //, and // i requested that...
4      i then notified the three credit agencies prev...
                             ...                        
995    when i received my  credit card statement the ...
996    i requested that i would like some type of ver...
997    tuesday, //2019 i received a call from a gentl...
998    on //2018, i invested money into     through a...
999     and transunion are reporting an incorrect cod...
Length: 1000, dtype: object

In [302]:
abstractive_1000 = abstractive_1000.abstractive.swifter.apply(lowering)

In [309]:
abstractive_1000

0      transworld systems is trying to collect a debt...
1      the company listed in this complaint does not ...
2      each violation is a statutory cost of $1000.00...
3      experian failed to send me a copy of the verif...
4      a bankruptcy and two accounts show discrepanci...
                             ...                        
995    the balance due was $120.00. i elected to use ...
996    the debt is still there, causing a negative ef...
997    a gentleman called to say he owed money from a...
998    i was scammed of $210.00 and never heard from ...
999    this account actually needs to be coded as an ...
Name: abstractive, Length: 1000, dtype: object

In [306]:
def stop_free(doc): 
  stop_words = set(stopwords.words('english'))
  stop_rem = " ".join([word for word in doc.split() if word not in stop_words])
  return stop_rem

In [307]:
extractive_1000_1 = extractive_1000.swifter.apply(stop_free)

In [308]:
extractive_1000_1

0                                                       
1      company right harass work want stop. extremely...
2      violations committed include limited : ( 1 ) v...
3      previously, //, //, // requested experian send...
4      notified three credit agencies previously stat...
                             ...                        
995    received credit card statement balance due {$1...
996    requested would like type verification suppose...
997    tuesday, //2019 received call gentleman name h...
998    //2018, invested money broker found online. ti...
999    transunion reporting incorrect coding / partia...
Length: 1000, dtype: object

In [310]:
abstractive_1000 = abstractive_1000.swifter.apply(stop_free)

In [311]:
abstractive_1000 = pd.DataFrame(abstractive_1000)

In [312]:
abstractive_1000.abstractive[995]

"balance due $120.00. elected use rewards '$130.00' make payment month left credit $5.00. also showed account late payment fee $25.00"

In [335]:
abstractive_1000['extractive'] = extractive_1000_1

In [336]:
original_1000 = original_cleaned3[0:1000]

In [337]:
abstractive_1000['original'] = original_cleaned3

In [338]:
df_compare = abstractive_1000
df_compare.head()

,abstractive,extractive,original
0,"transworld systems trying collect debt mine, o...",,transworld systems inc. trying collect debt mi...
1,company listed complaint right harass work. ex...,company right harass work want stop. extremely...,"past 2 weeks, receiving excessive amounts tele..."
2,"violation statutory cost $1000.00 each, includ...",violations committed include limited : ( 1 ) v...,pioneer committed several federal violations m...
3,experian failed send copy verifiable proof fil...,"previously, //, //, // requested experian send...","previously, xx/xx/xxxx, xx/xx/xxxx, xx/xx/xxxx..."
4,bankruptcy two accounts show discrepancies cre...,notified three credit agencies previously stat...,hello complaint three credit reporting compani...


There are some empty rows for extractive, because no extractive summaries were generated for one-sentence complaints. Let's remove them.

In [339]:
more_than_one_liners = df_compare.extractive != ''

more_than_one_liners

0      False
1       True
2       True
3       True
4       True
       ...  
995     True
996     True
997     True
998     True
999     True
Name: extractive, Length: 1000, dtype: bool

In [340]:
df_compare_full = df_compare[more_than_one_liners]

In [341]:
df_compare_full.head()

,abstractive,extractive,original
1,company listed complaint right harass work. ex...,company right harass work want stop. extremely...,"past 2 weeks, receiving excessive amounts tele..."
2,"violation statutory cost $1000.00 each, includ...",violations committed include limited : ( 1 ) v...,pioneer committed several federal violations m...
3,experian failed send copy verifiable proof fil...,"previously, //, //, // requested experian send...","previously, xx/xx/xxxx, xx/xx/xxxx, xx/xx/xxxx..."
4,bankruptcy two accounts show discrepancies cre...,notified three credit agencies previously stat...,hello complaint three credit reporting compani...
6,account 7 years old credit report. incorrect r...,account 7 years old therefore credit report. i...,xxxx reporting incorrectly equifax xxxx accoun...


Need to lower for extractive.

In [342]:
extractive_cl = [complaint.lower() for complaint in df_compare_full.extractive]

In [343]:
extractive = pd.Series(extractive_cl, index = df_compare_full.index)

extractive

1      company right harass work want stop. extremely...
2      violations committed include limited : ( 1 ) v...
3      previously, //, //, // requested experian send...
4      notified three credit agencies previously stat...
6      account 7 years old therefore credit report. i...
                             ...                        
995    received credit card statement balance due {$1...
996    requested would like type verification suppose...
997    tuesday, //2019 received call gentleman name h...
998    //2018, invested money broker found online. ti...
999    transunion reporting incorrect coding / partia...
Length: 929, dtype: object

In [344]:
df_compare_full = df_compare_full.drop('extractive', axis = 1)

df_compare_full.head()

,abstractive,original
1,company listed complaint right harass work. ex...,"past 2 weeks, receiving excessive amounts tele..."
2,"violation statutory cost $1000.00 each, includ...",pioneer committed several federal violations m...
3,experian failed send copy verifiable proof fil...,"previously, xx/xx/xxxx, xx/xx/xxxx, xx/xx/xxxx..."
4,bankruptcy two accounts show discrepancies cre...,hello complaint three credit reporting compani...
6,account 7 years old credit report. incorrect r...,xxxx reporting incorrectly equifax xxxx accoun...


In [345]:
df_compare_full['extractive'] = extractive

In [346]:
df_compare_full.head()

,abstractive,original,extractive
1,company listed complaint right harass work. ex...,"past 2 weeks, receiving excessive amounts tele...",company right harass work want stop. extremely...
2,"violation statutory cost $1000.00 each, includ...",pioneer committed several federal violations m...,violations committed include limited : ( 1 ) v...
3,experian failed send copy verifiable proof fil...,"previously, xx/xx/xxxx, xx/xx/xxxx, xx/xx/xxxx...","previously, //, //, // requested experian send..."
4,bankruptcy two accounts show discrepancies cre...,hello complaint three credit reporting compani...,notified three credit agencies previously stat...
6,account 7 years old credit report. incorrect r...,xxxx reporting incorrectly equifax xxxx accoun...,account 7 years old therefore credit report. i...


In [347]:
def odd_tokens_free(doc):
  import re
  processed = re.sub("(xx|xxxx|XX|XXXX|XX/XX/XXXX)","",doc)
  return processed

In [348]:
original_odd_free = df_compare_full.original.swifter.apply(odd_tokens_free)

In [349]:
df_compare_full = df_compare_full.drop('original', axis = 1)

df_compare_full.head()

,abstractive,extractive
1,company listed complaint right harass work. ex...,company right harass work want stop. extremely...
2,"violation statutory cost $1000.00 each, includ...",violations committed include limited : ( 1 ) v...
3,experian failed send copy verifiable proof fil...,"previously, //, //, // requested experian send..."
4,bankruptcy two accounts show discrepancies cre...,notified three credit agencies previously stat...
6,account 7 years old credit report. incorrect r...,account 7 years old therefore credit report. i...


In [350]:
df_compare_full['original'] = original_odd_free

df_compare_full.head()

,abstractive,extractive,original
1,company listed complaint right harass work. ex...,company right harass work want stop. extremely...,"past 2 weeks, receiving excessive amounts tele..."
2,"violation statutory cost $1000.00 each, includ...",violations committed include limited : ( 1 ) v...,pioneer committed several federal violations m...
3,experian failed send copy verifiable proof fil...,"previously, //, //, // requested experian send...","previously, //, //, // requested experian send..."
4,bankruptcy two accounts show discrepancies cre...,notified three credit agencies previously stat...,hello complaint three credit reporting compani...
6,account 7 years old credit report. incorrect r...,account 7 years old therefore credit report. i...,reporting incorrectly equifax account balanc...


# Eyeballing Some Key Examples

In [351]:
df_compare_full.head()

,abstractive,extractive,original
1,company listed complaint right harass work. ex...,company right harass work want stop. extremely...,"past 2 weeks, receiving excessive amounts tele..."
2,"violation statutory cost $1000.00 each, includ...",violations committed include limited : ( 1 ) v...,pioneer committed several federal violations m...
3,experian failed send copy verifiable proof fil...,"previously, //, //, // requested experian send...","previously, //, //, // requested experian send..."
4,bankruptcy two accounts show discrepancies cre...,notified three credit agencies previously stat...,hello complaint three credit reporting compani...
6,account 7 years old credit report. incorrect r...,account 7 years old therefore credit report. i...,reporting incorrectly equifax account balanc...


In [361]:
df_compare_full.original[500]

"//2018, day mother died, made purchase amount {$100.00}. due stress attention required settle mother 's estate, three days late paying amount full. asked leniency due circumstances. continued ask late fee reversed ; charged fees interest original late fee. getting least half dozen phone calls day - consider harrassment -  . paid balance owed. could easily forgiven original late fee. new account made one purchase it."

In [362]:
df_compare_full.extractive[500]

"//2018, day mother died, made purchase amount {$100. 00}. due stress attention required settle mother 's estate, three days late paying amount full. could easily forgiven original late fee. new account made one purchase it."

In [363]:
df_compare_full.abstractive[500]

"day mother died, made purchase amount $100.00. due stress attention required settle mother's estate, three days late paying sum full - asked leniency circumstances dictated ; charged fees interest original late fee!"

# Cosine Similarity between Abstractive and Extractive

In [364]:
df_compare_full.abstractive[1]

"company listed complaint right harass work. extremely distracting told 5 times day excessive amount phone calls collection agency, says michael o'neill-mcdonald jr."

In [365]:
df_compare_full.extractive[1]

'company right harass work want stop. extremely distracting told 5 times day call collection agency work.'

In [366]:
test_documents = list()

test_documents.append(df_compare_full.abstractive[1])
test_documents.append(df_compare_full.extractive[1])

test_documents

["company listed complaint right harass work. extremely distracting told 5 times day excessive amount phone calls collection agency, says michael o'neill-mcdonald jr.",
 'company right harass work want stop. extremely distracting told 5 times day call collection agency work.']

In [367]:
# Create the Document Term Matrix
tfidf_vectorizer = TfidfVectorizer()
sparse_matrix = tfidf_vectorizer.fit_transform(test_documents)

In [368]:
print(cosine_similarity(sparse_matrix))

[[1.         0.46990184]
 [0.46990184 1.        ]]


In [369]:
cosine_similarity(sparse_matrix)[0][1]

0.46990184423194137

In [370]:
df_compare_full.index

Int64Index([  1,   2,   3,   4,   6,   9,  10,  11,  12,  13,
            ...
            990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
           dtype='int64', length=929)

In [371]:
scores = []

In [372]:
for row in tqdm(df_compare_full.index):
  abstractive_compare = df_compare_full.abstractive[row]
  extractive_compare = df_compare_full.extractive[row]
  compare_documents = list()
  compare_documents.append(abstractive_compare)
  compare_documents.append(extractive_compare)
  tfidf_vectorizer = TfidfVectorizer(stop_words='english')
  tfidf_vectorizer = TfidfVectorizer()
  sparse_matrix = tfidf_vectorizer.fit_transform(compare_documents)
  single_score_row = cosine_similarity(sparse_matrix)[0][1]
  scores.append(single_score_row)

100%|██████████| 929/929 [00:02<00:00, 442.44it/s]


In [373]:
scores_series = pd.Series(scores)

In [374]:
scores_series.describe()

count    929.000000
mean       0.393145
std        0.175121
min        0.000000
25%        0.275937
50%        0.386838
75%        0.499941
max        1.000000
dtype: float64

# Soft Cosine Similarity between Abstractive and Extractive

https://notebook.community/piskvorky/gensim/docs/notebooks/soft_cosine_tutorial 

In [375]:
import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)
# Download the FastText model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300') #https://fasttext.cc/docs/en/pretrained-vectors.html#

3.8.3


In [376]:
abs_list = df_compare_full.abstractive.tolist()

In [377]:
ext_list = df_compare_full.extractive.tolist()

In [378]:
documents = abs_list + ext_list

In [379]:
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

In [380]:
from gensim.models.keyedvectors import WordEmbeddingSimilarityIndex
from gensim.similarities.termsim import SparseTermSimilarityMatrix

# Prepare the similarity matrix
similarity_index = WordEmbeddingSimilarityIndex(fasttext_model300)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary)

In [381]:
test_sent_1 = dictionary.doc2bow(simple_preprocess(df_compare_full.abstractive[1]))

In [382]:
test_sent_2 = dictionary.doc2bow(simple_preprocess(df_compare_full.extractive[1]))

In [383]:
print(similarity_matrix.inner_product(test_sent_1, test_sent_2, normalized=True))

0.6817356


In [384]:
# Convert all abstractive summarized complaints into bag-of-words vectors.
bag_of_words_abs_list = []
for complaint in abs_list:
    processed_abs_sent = dictionary.doc2bow(simple_preprocess(complaint))
    bag_of_words_abs_list.append(processed_abs_sent)

In [385]:
bag_of_words_abs_list[928]

[(40, 1),
 (44, 1),
 (47, 2),
 (53, 1),
 (65, 1),
 (75, 1),
 (107, 1),
 (108, 1),
 (110, 1),
 (824, 1),
 (991, 1),
 (1335, 1),
 (2212, 1),
 (2213, 1)]

In [386]:
# Convert all extractive summarized complaints into bag-of-words vectors.
bag_of_words_ext_list = []
for complaint in ext_list:
    processed_ext_sent = dictionary.doc2bow(simple_preprocess(complaint))
    bag_of_words_ext_list.append(processed_ext_sent)

In [387]:
bag_of_words_ext_list[928]

[(50, 1),
 (65, 1),
 (75, 2),
 (76, 1),
 (91, 1),
 (108, 1),
 (140, 1),
 (257, 1),
 (260, 1),
 (261, 1),
 (265, 1),
 (349, 2),
 (396, 1),
 (824, 1),
 (991, 1),
 (1499, 1),
 (2212, 1),
 (2213, 1),
 (2328, 1),
 (2473, 1)]

## Now for all

In [388]:
score_list = []
for i in range (0, 929):
    scores = similarity_matrix.inner_product(bag_of_words_abs_list[i], bag_of_words_ext_list[i], normalized=True)
    score_list.append(scores)

In [389]:
score_series = pd.Series(score_list)

In [390]:
score_series.describe()

count    929.000000
mean       0.590324
std        0.171344
min        0.000000
25%        0.490324
50%        0.605708
75%        0.710848
max        1.000000
dtype: float64

# Soft Cosine Similarity between Abstractive and Original

In [391]:
abs_list = df_compare_full.abstractive.tolist()

In [392]:
org_list = df_compare_full.original.tolist()

In [393]:
documents_comp = abs_list + org_list

In [394]:
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents_comp])

In [395]:
from gensim.models.keyedvectors import WordEmbeddingSimilarityIndex
from gensim.similarities.termsim import SparseTermSimilarityMatrix

# Prepare the similarity matrix
similarity_index = WordEmbeddingSimilarityIndex(fasttext_model300)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary)

In [396]:
test_sent_1 = dictionary.doc2bow(simple_preprocess(df_compare_full.abstractive[1]))

In [397]:
test_sent_2 = dictionary.doc2bow(simple_preprocess(df_compare_full.original[1]))

In [398]:
print(similarity_matrix.inner_product(test_sent_1, test_sent_2, normalized=True))

0.7594595


In [399]:
# Convert all abstractive summarized complaints into bag-of-words vectors.
bag_of_words_abs_list = []
for complaint in abs_list:
    processed_abs_sent = dictionary.doc2bow(simple_preprocess(complaint))
    bag_of_words_abs_list.append(processed_abs_sent)

In [400]:
bag_of_words_abs_list[928]

[(40, 1),
 (44, 1),
 (47, 2),
 (53, 1),
 (65, 1),
 (75, 1),
 (107, 1),
 (108, 1),
 (110, 1),
 (824, 1),
 (991, 1),
 (1335, 1),
 (2212, 1),
 (2213, 1)]

In [401]:
# Convert all extractive summarized complaints into bag-of-words vectors.
bag_of_words_org_list = []
for complaint in org_list:
    processed_org_sent = dictionary.doc2bow(simple_preprocess(complaint))
    bag_of_words_org_list.append(processed_org_sent)

In [402]:
bag_of_words_org_list[928]

[(40, 1),
 (44, 1),
 (47, 3),
 (50, 2),
 (53, 1),
 (65, 2),
 (75, 2),
 (76, 2),
 (91, 2),
 (94, 2),
 (107, 1),
 (108, 2),
 (110, 1),
 (114, 1),
 (140, 1),
 (257, 1),
 (260, 1),
 (261, 2),
 (265, 1),
 (349, 2),
 (396, 1),
 (574, 1),
 (824, 1),
 (991, 1),
 (1216, 1),
 (1499, 1),
 (2212, 1),
 (2213, 2),
 (2328, 1),
 (2473, 1),
 (2925, 1)]

## Now for all

In [403]:
score_list = []
for i in range (0, 929):
    scores = similarity_matrix.inner_product(bag_of_words_abs_list[i], bag_of_words_org_list[i], normalized=True)
    score_list.append(scores)

In [404]:
score_series = pd.Series(score_list)

In [405]:
score_series.describe()

count    929.000000
mean       0.687210
std        0.121787
min        0.108089
25%        0.620937
50%        0.698846
75%        0.766243
max        1.000000
dtype: float64

# Soft Cosine Similarity between Extractive (all) and Original

In [406]:
original_all = original_cleaned3

In [407]:
def full_stop_space(doc):
  import re
  rx = r"\.(?=\S)"
  result = re.sub(rx, ". ", doc)
  return result

def odd_tokens_free(doc):
  import re
  processed = re.sub("(xx|xxxx|XX|XXXX|XX/XX/XXXX)","",doc)
  return processed

def lowering(doc):
    lowered = "".join([sentence.lower() for sentence in doc])
    return lowered

def stop_free(doc): 
  stop_words = set(stopwords.words('english'))
  stop_rem = " ".join([word for word in doc.split() if word not in stop_words])
  return stop_rem

In [413]:
extractive_all_unclean = pd.Series(extractive)

In [414]:
extractive_clean1 = extractive_all_unclean.progress_apply(full_stop_space)

In [416]:
extractive_clean2 = extractive_clean1.progress_apply(odd_tokens_free)

In [418]:
extractive_clean3 = extractive_clean2.progress_apply(lowering)

In [420]:
extractive_clean4 = extractive_clean3.progress_apply(stop_free)

In [436]:
extractive_all = extractive_clean4

In [437]:
df_tocompare = pd.DataFrame(extractive_all)

In [438]:
df_tocompare['original_all'] = original_all

In [439]:
df_tocompare

,0,original_all
0,,transworld systems inc. trying collect debt mi...
1,company right harass work want stop. extremely...,"past 2 weeks, receiving excessive amounts tele..."
2,violations committed include limited : ( 1 ) v...,pioneer committed several federal violations m...
3,"previously, //, //, // requested experian send...","previously, xx/xx/xxxx, xx/xx/xxxx, xx/xx/xxxx..."
4,notified three credit agencies previously stat...,hello complaint three credit reporting compani...
...,...,...
657714,fine print supposedly n't see final payment mu...,automatic payment car loan. fine print suppose...
657715,hold 10 minutes automated system recording tol...,recieved collections call unknown company morn...
657716,", 2015, contacted , branch manager ( # ) gatew...","xxxx xxxx, 2015, contacted xxxx xxxx, branch m..."
657717,,"get chase services mortgage, owns original loa..."


In [447]:
df_tocompare = df_tocompare.rename(columns = {0:'extractive_all'})

In [452]:
df_tocompare

,extractive_all,original_all
0,,transworld systems inc. trying collect debt mi...
1,company right harass work want stop. extremely...,"past 2 weeks, receiving excessive amounts tele..."
2,violations committed include limited : ( 1 ) v...,pioneer committed several federal violations m...
3,"previously, //, //, // requested experian send...","previously, xx/xx/xxxx, xx/xx/xxxx, xx/xx/xxxx..."
4,notified three credit agencies previously stat...,hello complaint three credit reporting compani...
...,...,...
657714,fine print supposedly n't see final payment mu...,automatic payment car loan. fine print suppose...
657715,hold 10 minutes automated system recording tol...,recieved collections call unknown company morn...
657716,", 2015, contacted , branch manager ( # ) gatew...","xxxx xxxx, 2015, contacted xxxx xxxx, branch m..."
657717,,"get chase services mortgage, owns original loa..."


In [453]:
more_than_one_liners = df_tocompare.extractive_all != ''

more_than_one_liners

df_tocompare_full = df_tocompare[more_than_one_liners]

In [454]:
df_tocompare_full

,extractive_all,original_all
1,company right harass work want stop. extremely...,"past 2 weeks, receiving excessive amounts tele..."
2,violations committed include limited : ( 1 ) v...,pioneer committed several federal violations m...
3,"previously, //, //, // requested experian send...","previously, xx/xx/xxxx, xx/xx/xxxx, xx/xx/xxxx..."
4,notified three credit agencies previously stat...,hello complaint three credit reporting compani...
6,account 7 years old therefore credit report. i...,xxxx reporting incorrectly equifax xxxx accoun...
...,...,...
657713,could qualifying year known.,"life student loans xxxx xxxx navient, continuo..."
657714,fine print supposedly n't see final payment mu...,automatic payment car loan. fine print suppose...
657715,hold 10 minutes automated system recording tol...,recieved collections call unknown company morn...
657716,", 2015, contacted , branch manager ( # ) gatew...","xxxx xxxx, 2015, contacted xxxx xxxx, branch m..."


In [457]:
ext_list = df_tocompare_full.extractive_all.tolist()

In [458]:
org_list = df_tocompare_full.original_all.tolist()

In [459]:
documents_comp = ext_list + org_list

In [460]:
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents_comp])

In [461]:
from gensim.models.keyedvectors import WordEmbeddingSimilarityIndex
from gensim.similarities.termsim import SparseTermSimilarityMatrix

# Prepare the similarity matrix
similarity_index = WordEmbeddingSimilarityIndex(fasttext_model300)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary)

In [472]:
import pickle

with open('all_ext_org_similarity_matrix.pkl', 'wb') as fp:
    pickle.dump(similarity_matrix, fp)

In [462]:
# Convert all extractive summarized complaints into bag-of-words vectors.
bag_of_words_org_list = []
for complaint in org_list:
    processed_org_sent = dictionary.doc2bow(simple_preprocess(complaint))
    bag_of_words_org_list.append(processed_org_sent)

In [467]:
# Convert all extractive summarized complaints into bag-of-words vectors.
bag_of_words_ext_list = []
for complaint in ext_list:
    processed_ext_sent = dictionary.doc2bow(simple_preprocess(complaint))
    bag_of_words_ext_list.append(processed_ext_sent)

## Now for all

In [469]:
score_list = []
for i in range (0, 596913):
    scores = similarity_matrix.inner_product(bag_of_words_ext_list[i], bag_of_words_org_list[i], normalized=True)
    score_list.append(scores)

In [470]:
score_series = pd.Series(score_list)

In [471]:
score_series.describe()

count    596913.000000
mean          0.684405
std           0.191578
min           0.000000
25%           0.573873
50%           0.728940
75%           0.832852
max           1.000000
dtype: float64